<a href="https://colab.research.google.com/github/crazylazylife/galaxy_zoo_classification_project/blob/master/galaxy_zoo_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
!pip install -q kaggle

In [0]:
%cd ../root

/root


In [0]:
!mkdir .kaggle
%cd .kaggle
uploading = files.upload()

mkdir: cannot create directory ‘.kaggle’: File exists
/root/.kaggle


Saving kaggle.json to kaggle.json


In [0]:
%cd ../../content

/content


In [0]:
!kaggle competitions download -c galaxy-zoo-the-galaxy-challenge

  0% 0.00/265k [00:00<?, ?B/s]
100% 265k/265k [00:00<00:00, 35.0MB/s]
  0% 0.00/265k [00:00<?, ?B/s]
100% 265k/265k [00:00<00:00, 79.3MB/s]
  0% 0.00/520k [00:00<?, ?B/s]
100% 520k/520k [00:00<00:00, 34.2MB/s]
100% 790M/792M [00:08<00:00, 50.3MB/s]
100% 792M/792M [00:08<00:00, 95.1MB/s]
 99% 1.00G/1.01G [00:11<00:00, 48.5MB/s]
100% 1.01G/1.01G [00:11<00:00, 91.7MB/s]
  0% 0.00/4.63M [00:00<?, ?B/s]
100% 4.63M/4.63M [00:00<00:00, 42.4MB/s]


In [0]:
!ls

all_ones_benchmark.zip	     images_training_rev1.zip
all_zeros_benchmark.zip      sample_data
central_pixel_benchmark.zip  training_solutions_rev1.zip
images_test_rev1.zip


In [0]:
!unzip training_solutions_rev1.zip
!unzip images_training_rev1.zip

-----------------------------------------------------------------------------
Beginning preprocessing, developing and training the ResNet model for the task of classification

In [1]:
#Importing the libraries
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, ZeroPadding2D, Flatten, AveragePooling2d, MaxPolling2D, Conv2D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2d, Dense, Dropout
from keras.activations import relu, softmax
from keras.models import Model, load_model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.initializers import glorot_uniform


import keras.backend as K

from matplotlib.pyplot import imshow
%matplotlib inline

Using TensorFlow backend.


In [0]:
#Image preprocessing

In [0]:
#Building the ResNet Architecture

class ResNet:
  
  def identity_block(X, f_shape, n_filters, stage, block_name):
    '''
    X -> Input tensor 
    f_shape -> Integer, denoting the shape of the middle filter
    n_filters -> List, denoting the number of filters in each layer
    stage -> Integer, denoting the name of the layer
    block_name -> Denotes the name of the layer
    '''
    
    
    f1, f2, f3 = n_filters
    X_skip = X
    
    #1st component
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block+"_branch_2a")(X)
    X = Activation("relu")(X)
    
    #2nd component
    X = Conv2D(filters=f2, kernel_size=(f, f), strides=(1, 1), padding="same", name= "resnet"+str(stage)+block_name+"_branch_2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block+"_branch_2b")(X)
    X = Activation("relu")(X)
    
    #3rd component
    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block+"_branch_2c")(X)
    
    #Inserting skip connection
    X = Add()([X, X_skip])
    X = Activation("relu")(X)
    
    return X
  
  def convolutional_block(X, f_shape, n_filters, stage, block_name, s=2):
    '''
    X -> Input tensor 
    f_shape -> Integer, denoting the shape of the middle filter
    n_filters -> List, denoting the number of filters in each layer
    stage -> Integer, denoting the name of the layer
    block_name -> Denotes the name of the layer
    s -> denoting the stride to be used
    '''
    
    f1, f2, f3 = n_filters
    X_skip = X
    
    #1st component
    #1st component
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=(s, s), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_2a", kernel_initializer=glorot_uniform(seed=0))(X), 
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block+"_branch_2a")(X)
    X = Activation("relu")(X)
    
    #2nd component
    X = Conv2D(filters=f2, kernel_size=(f, f), strides=(1, 1), padding="same", name= "resnet"+str(stage)+block_name+"_branch_2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block+"_branch_2b")(X)
    X = Activation("relu")(X)
    
    #3rd component
    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name = "batchnorm"+str(stage)+block+"_branch_2c")(X)
    
    #Convolution for the shortcut
    X_skip = Conv2D(filters=f3, kernel_size=(1, 1), strides=(s, s), padding="valid", name= "resnet"+str(stage)+block_name+"_branch_1", kernel_initializer=glorot_uniform(seed=0))(X)
    X_skip = BatchNormalization(axis=3, name= "batchnorm"+str(stage)+block+"_branch_1")(X_skip)
    
    #Inserting skip connection
    X = Add()([X, X_skip])
    X = Activation("relu")(X)
    
    return X
  
  def ResNet18(input_shape=(224, 224, 3), classes=37):
    '''
    Defining the ResNet18 Architecture used
    '''